In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint
from gensim.models import Word2Vec 

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os

# from keras.utils import multi_gpu_model

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import keras
feature_len = 768
max_go_len = 256
max_seq_len = 512
 
dataset_name = 'SHS27k'
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di
 
protein2go =  load_dict('ensemble2go_terms_'+dataset_name+'.pkl')

prot2emb = {}
 
for key, value in protein2go.items():
    X_go1 =  np.random.randn(1,768)
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    for  go in  allgos:
        if go.startswith('GO') and os.path.exists('../ncbi_allfeatures4go/'+go+'_0.npy'):
            feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
            
            if count + feature.shape[0] > max_go_len:
                print(count + feature.shape[0])
                break
            X_go1 = np.concatenate((X_go1,feature ))    
            count += feature.shape[0]
        else:
            feature = np.random.randn(1,768)
            if count + feature.shape[0] > max_go_len:
                print(count + feature.shape[0])
                break
            X_go1 = np.concatenate((X_go1,feature ))    
            count += feature.shape[0]
            
    
    prot2emb[key] =  X_go1[1:]   



In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
         
        self.protein2go =  load_dict('ensemble2go_terms_'+dataset_name+'.pkl')
        self.protein2seq = load_dict('ensemble2seqs_'+dataset_name+'.pkl')
         
         
        self.protein2onehot = {}
        self.onehot_seqs()
         
        self.data = np.load(ppi_pair_file)
        self.ensp1 = self.data['ensp1']
        self.ensp2 = self.data['ensp2']
        self.labels = self.data['labels']
        self.on_epoch_end()
    
      
     
            
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 

                
    def project2gpu_go(self, protemb_tmp):
        X_go1 = np.zeros((self.max_golen,768))
        protemb_len = protemb_tmp.shape[0]
        X_go1[:protemb_len] = protemb_tmp
        return X_go1

    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.labels.shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

     
            
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.labels.shape[0])
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        
        y = self.labels[list_IDs_temp]
        
        X_seq1 = np.array([self.protein2onehot[p1] for p1 in self.ensp1[list_IDs_temp]])
        X_seq2 = np.array([self.protein2onehot[p2] for p2 in self.ensp2[list_IDs_temp]])
        X_go1 =  np.array([self.project2gpu_go(prot2emb[p1]) for p1 in self.ensp1[list_IDs_temp]])
        X_go2 =  np.array([self.project2gpu_go(prot2emb[p2]) for p2 in self.ensp2[list_IDs_temp]])

#         X_go1 = np.empty((self.batch_size, self.max_golen,768))
#         X_go2 = np.empty((self.batch_size, self.max_golen,768))
#         y = np.empty((self.batch_size, 7))
#         X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
#         X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        
        
         

#         # Generate data
#         for i, ID in enumerate(list_IDs_temp):
             
#             p1 = self.ensp1[ID]
#             p2 = self.ensp2[ID]
#             y[i] = self.labels[ID]
#             prot1emb_tmp = prot2emb[p1]
#             X_go1[i,:prot1emb_tmp.shape[0]] = prot1emb_tmp
            
#             prot2emb_tmp = prot2emb[p2]
#             X_go2[i,:prot2emb_tmp.shape[0]] = prot2emb_tmp
            
#             X_seq1[i] =  self.protein2onehot[p1]
#             X_seq2[i] =  self.protein2onehot[p2]
            
             
            
            
            
     
        return [X_go1,X_go2,    X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        X_go1 = np.empty((len(list_IDs_temp), self.max_golen,768))
         
        X_go2 = np.empty((len(list_IDs_temp), self.max_golen,768))
        y = np.empty((len(list_IDs_temp),7))
        
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        
        
         

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            p1 = self.ensp1[ID]
            p2 = self.ensp2[ID]
            y[i] = self.labels[ID]
            
            prot1emb_tmp = prot2emb[p1]
            X_go1[i,:prot1emb_tmp.shape[0]] = prot1emb_tmp
            
            prot2emb_tmp = prot2emb[p2]
            X_go2[i,:prot2emb_tmp.shape[0]] = prot2emb_tmp
            
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            
            
            
  
        return [X_go1,X_go2,   X_seq1, X_seq2] ,  y

In [5]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [6]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, dot, Flatten, CuDNNLSTM, Add
from keras.layers.merge import concatenate
from keras.optimizers import Adam,  RMSprop
from keras import regularizers
from keras_radam import RAdam
from keras_lookahead import Lookahead

def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 1, activation="relu",    padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu",   padding='same')(y)
#     x1 = Dropout(0.3)(x1)
    y = Conv1D(con1d_filters, 1, activation="relu",   padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu",   padding='same')(y)
#     x2 = Dropout(0.3)(x2)

    x3 = Conv1D(con1d_filters, 3, activation="relu",  padding='same')(input_tensor)
#     x3 = Dropout(0.3)(x3)

    x4 = Conv1D(con1d_filters, 1, activation="relu",  padding='same')(input_tensor)
#     x4 = Dropout(0.3)(x4)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(y)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y


 
def build_cnn_gru_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(gru_units, return_sequences=True))(input_x)
    x_gru = Dropout(0.3)(x_gru)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    
#     x_conv = Concatenate()([x_a, x_b,  x_c])
#     x_conv = Dense(128, activation='relu')(x_conv)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
#     x_gru = Concatenate()([x_gru_a, x_gru_b,  x_gru_c])
#     x_gru = Dense(128, activation='relu')(x_gru)
#     x = Concatenate()([x_conv, x_gru])
    
    
    
    
    
    x = Concatenate()([x_a, x_b, x_c, x_gru_a, x_gru_b, x_gru_c])
    x = Dense(256, activation='relu')(x)
    
    
    
#     x_conv = Concatenate()([x_a, x_b, x_c, ])
#     x_gru =  Concatenate()([x_gru_a, x_gru_b, x_gru_c ])
#     x = BatchNormalization()(x)
    
#     x_conv = Dense(128, activation = 'relu')(x_conv)
#     x_gru = Dense(128)(x_conv)
    return x


def build_fc_model(input_x):
     
    
    x_a = GlobalAveragePooling1D()(input_x)
    x_b = GlobalMaxPooling1D()(input_x)
    x_c = Attention()(input_x)
    x = Concatenate()([x_a, x_b, x_c])
    x = Dense(1024)(x)
    x = Dense(256)(x)
    return x 


def build_model():
    con_filters = 256
    gru_units = 64
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    
     
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    
    left_x_go = build_fc_model(left_input_go)
    right_x_go = build_fc_model(right_input_go)
    
#     left_x_go = build_cnn_gru_model(left_input_go, con_filters, gru_units)
#     right_x_go = build_cnn_gru_model(right_input_go, con_filters,gru_units)
    
    left_x_seq = build_cnn_gru_model(left_input_seq, con_filters//4, gru_units)
    right_x_seq = build_cnn_gru_model(right_input_seq, con_filters//4, gru_units)
    
      
   
    x =   Concatenate()([left_x_go  , right_x_go,   left_x_seq, right_x_seq])
    
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(7)(x)
#     x = BatchNormalization()(x)
    output = Activation('softmax')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_go, right_input_go,    left_input_seq, right_input_seq], output)
#     model = multi_gpu_model(model)
#     adam = Adam(lr=0.01, amsgrad=True, epsilon=1e-5)
    optimizer = Lookahead(RAdam())


    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


 

model = build_model()
model.summary()
# siamese_a = create_share_model()
# siamese_a.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512, 20)      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 512, 20)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 512, 16)      336         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 512, 16)      336         input

In [7]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

In [8]:

for rep in range(1):
    n_splits = 10
     
    count = 0
    num_total = 0.
    num_hit = 0.
    for split in range(n_splits):
        train_pairs_file = 'CV/'+ dataset_name + '_train'+str(split)+'.npz'
        test_pairs_file = 'CV/' + dataset_name +'_test'+str(split)+'.npz'
        valid_pairs_file = 'CV/'+ dataset_name +'_valid'+str(split)+'.npz'
        
         

        batch_size = 256
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
        model = build_model()
        

        save_model_name = 'CV/27_fc_go_seq_rand'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_acc', patience=20, verbose=0, mode='max')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True,verbose=1, monitor='val_acc', mode='max', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
#         validation_data = valid_generator,
#                                  callbacks=[earlyStopping, save_checkpoint]
         
        hist = model.fit_generator(generator=train_generator,
                   epochs = 100,verbose=1, validation_data = valid_generator, callbacks=[earlyStopping, save_checkpoint])
#         model.save_weights(save_model_name)
    

        model.load_weights(save_model_name)
        data = np.load(test_pairs_file)
         

        test_len = data['labels'].shape[0]
        list_IDs_temp = np.arange(test_len)
        y_true = np.zeros((test_len,7))
        y_pred_prob = np.zeros((test_len,7))
        for index in range(int(np.floor(test_len/batch_size)) +1):
            start_ind = index * batch_size
            if (index+1)* batch_size >= test_len:
                end_ind = test_len
            else:
                end_ind = (index+1)* batch_size
            indexes = list_IDs_temp[start_ind:end_ind]
            test_x, y_test = test_generator.all_data(indexes)
            y_pred_prob_bs = model.predict(test_x)
            y_true[start_ind:end_ind] = y_test.reshape(len(indexes),7)
            y_pred_prob[start_ind:end_ind] = y_pred_prob_bs
            del test_x
            del y_test
#         np.save('27_results'+str(split), y_pred_prob)
        y_test = y_true

#         test_x, y_test = test_generator.all_data(list_IDs_temp)
#         results = model.evaluate(test_x, y_test, batch_size=128)
#         print('test loss, test acc:', results)
#         y_pred_prob = model.predict(test_x)
        
        for i in range( y_test.shape[0]  ):
            num_total += 1
            if np.argmax(y_test[i]) == np.argmax(y_pred_prob[i]):
                num_hit += 1
        
        print(num_hit / num_total)
        del model
        del train_generator
        del valid_generator
        del test_generator
        del y_test
    accuracy = num_hit / num_total
    print (accuracy)

   




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
85/85 [==============================] - 87s 1s/step - loss: 1.8143 - acc: 0.2265 - val_loss: 1.6425 - val_acc: 0.3108

Epoch 00001: val_acc improved from -inf to 0.31076, saving model to CV/27_fc_go_seq_rand0-0.hdf5
Epoch 2/100
85/85 [==============================] - 76s 898ms/step - loss: 1.6040 - acc: 0.3040 - val_loss: 1.4665 - val_acc: 0.3720

Epoch 00002: val_acc improved from 0.31076 to 0.37196, saving model to CV/27_fc_go_seq_rand0-0.hdf5
Epoch 3/100
85/85 [==============================] - 85s 999ms/step - loss: 1.5142 - acc: 0.3390 - val_loss: 1.4330 - val_acc: 0.3863

Epoch 00003: val_acc improved from 0.37196 to 0.38628, saving model to CV/27_fc_go_seq_rand0-0.hdf5
Epoch 4/100
85/85 [==============================] - 78s 914ms/step - loss: 1.4335 - acc: 0.3720 - val_loss: 1.3654 - val_acc: 0.4171

Epoch 00004: val_acc improved from 0.38628 to 0.41710, saving model to C

85/85 [==============================] - 70s 819ms/step - loss: 0.9415 - acc: 0.5531 - val_loss: 0.9944 - val_acc: 0.5369

Epoch 00040: val_acc improved from 0.53125 to 0.53689, saving model to CV/27_fc_go_seq_rand0-0.hdf5
Epoch 41/100
85/85 [==============================] - 71s 830ms/step - loss: 0.9670 - acc: 0.5485 - val_loss: 1.1042 - val_acc: 0.4905

Epoch 00041: val_acc did not improve from 0.53689
Epoch 42/100
85/85 [==============================] - 70s 822ms/step - loss: 0.9646 - acc: 0.5450 - val_loss: 1.0024 - val_acc: 0.5321

Epoch 00042: val_acc did not improve from 0.53689
Epoch 43/100
85/85 [==============================] - 71s 831ms/step - loss: 0.9618 - acc: 0.5465 - val_loss: 1.0381 - val_acc: 0.5148

Epoch 00043: val_acc did not improve from 0.53689
Epoch 44/100
85/85 [==============================] - 71s 837ms/step - loss: 0.9113 - acc: 0.5597 - val_loss: 1.0516 - val_acc: 0.5208

Epoch 00044: val_acc did not improve from 0.53689
Epoch 45/100
85/85 [=============

0.5929499072356216


KeyboardInterrupt: 

In [ ]:
# del model
# del train_generator
# del valid_generator
# del test_generator
# del test_x
# del y_test

In [ ]:
from sklearn.metrics import confusion_matrix
y_true2 = np.argmax(y_true, axis=1)
y_pred = np.argmax(y_pred_prob, axis=1)
confusion_matrix(y_true2, y_pred)